<a href="https://colab.research.google.com/github/jodfedlet/TCC/blob/main/TCCII/Twitter/preprocessing_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Objetivos
- Visualizar os tweets antes e depois do pré-processamento
  - Quantidade de tweets - quantidade de plavras por tweets e média de palavras dos tweets geral/por ano e por mês
- Pre-processar os tweets:
  - remoção de stop words 
  - remoção de URLs
  - conversão dos textos em letras minúsculas
  - remoção de pontuação e acentuação
  - remocação de palavras com menos:
      - que iniciam com os caracteres @ ou #
      - que possuem menos de 3 caracteres que são diferentes de ht ( sigla do país haiti)
      - palavras que possuem mais de 3 occorências seguidas ( ex: kkkkkk)
  - aplicação de bigrams ( técnica que faz com que palavras que aparecem juntas permaneçam juntas. Ex: Rio de Janeiro = rio_janeiro)
  - lematização dos tweets (processo no qual é realizada uma análise morfológica das palavras, por
exemplo palavras da terceira posição são alteradas para a primeira e verbos no passado e
futuro transformados no presente)

###Importando/instalando bibliotecas

In [ ]:
!pip install unidecode
!python3 -m spacy download pt_core_news_sm
!pip install gensim

In [57]:

import pandas as pd
import numpy as np
import nltk
import re
import spacy
import gensim
import matplotlib as mpl
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from google.colab import drive
drive.mount("/content/drive")
nltk.download('all-corpora')
nltk.download('punkt')
nltk.download('rslp')

from nltk.corpus import stopwords
from unidecode import unidecode
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk import regexp_tokenize
from gensim.models import Phrases


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading collection 'all-corpora'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloadin

# Carregamento dos tweets

In [127]:
try:
  tweets_orig = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TCC/csv/all_tweets_with_retweets.csv', lineterminator='\n', index_col=0)
  # tweets_orig['date'] = pd.to_datetime(tweets_orig.datetime).dt.strftime("%Y-%m-%d")
  # tweets_orig = tweets_orig.set_index('date')
  # tweets_orig = tweets_orig.drop(columns="datetime")
except Exception as e:
  print(e)

In [128]:
tweets_orig.shape

(185348, 4)

In [129]:
tweets_orig.head()

,datetime,tweet_id,user_id,text
0,2022-05-30 23:45:05+00:00,1531421471146393601,1217204834211958785,@geglobo Paquetá é haitiano
1,2022-05-30 23:00:53+00:00,1531410348141592577,1473798922472767488,@BetoLuiz_RU @jonesmanoel_PCB @deccache O Hait...
2,2022-05-30 22:57:20+00:00,1531409451122577410,1521912766151417856,O senhor vai investir na educação da Venezuela...
3,2022-05-30 22:55:14+00:00,1531408925945434112,1253882714723409920,"@rsallesmma Que nada, na mão de vocês já tá vi..."
4,2022-05-30 22:55:14+00:00,1531408923277807617,40320768,@JoppertLeonardo @jonesmanoel_PCB @deccache En...


#Estatística dos tweets

In [130]:
def tweets_stats(df, option):
  df['datetime'] = pd.to_datetime(df.datetime, format='%Y-%m-%d')

  def get_df_len__count_words__mean(df):
    count_all_tweets = len(df)

    if option == 'pre':
      df_text = df['text_preprocessed']
    else:
       df_text = df['text']

    df['count_word'] = df_text.str.split().str.len()
    count_all_words = df['count_word'].sum()
    mean = round(count_all_words / count_all_tweets, 2)
    return (df, count_all_tweets, count_all_words, mean)

  #data by year
  def plot_by_year(): 
    by_year = df.groupby([df['datetime'].dt.year.rename('Year')]).agg({'tweet_id':['count'], 'count_word': ['sum']}).reset_index()
    by_year.columns = ['Year', 'Count tweets', 'Count words']
    by_year = by_year.assign(Mean_words = lambda x: round(x['Count words'] / x['Count tweets'], 2))
    print(by_year)

  #data by 3 month
  def quartely(): 
    trimestral = df.groupby(pd.Grouper(key='datetime', freq='3M', closed='left')).agg({'tweet_id':['count'], 'count_word': ['sum']}).reset_index()
    trimestral.columns = ['Date', 'Count tweets', 'Count words']
    trimestral = trimestral.assign(Mean_words = lambda x: round(x['Count words'] / x['Count tweets'], 2))
    print(trimestral)

  #by year month
  def plot_by_year_and_month():
    by_year_month = df.groupby([df['datetime'].dt.year.rename('Year'), df['datetime'].dt.month.rename('Month')]).agg({'tweet_id':['count'], 'count_word': ['sum']}).reset_index()
    by_year_month.columns = ['Year', 'Month', 'Count tweets', 'Count words']
    by_year_month = by_year_month.assign(Mean_words = lambda x: round(x['Count words'] / x['Count tweets'], 2))
    print(by_year_month)
   
  df, df_len, count_words, mean_words  = get_df_len__count_words__mean(df)
  # print('Total: {0}, Count word: {1}, Mean words: {2}'.format(df_len,count_words, mean_words))
  # print()

  print('*****Agrupado por ano****')
  plot_by_year()
  print('\n')
  
  print('*****Agrupamento trimestral****')
  quartely()
  print('\n')

  print('*****Agrupado por mês e ano****')
  plot_by_year_and_month()

###Estatística dos tweets antes do pré-processamento:

In [131]:
tweets_stats(tweets_orig, 'ori')

*****Agrupado por ano****
   Year  Count tweets  Count words  Mean_words
0  2018         23691       531047       22.42
1  2019         29739       742523       24.97
2  2020         56375      1317064       23.36
3  2021         62625      1621237       25.89
4  2022         12918       330690       25.60


*****Agrupamento trimestral****
                        Date  Count tweets  Count words  Mean_words
0  2018-03-31 00:00:00+00:00          6356       138675       21.82
1  2018-06-30 00:00:00+00:00          5551       124146       22.36
2  2018-09-30 00:00:00+00:00          5944       129531       21.79
3  2018-12-31 00:00:00+00:00          5792       137499       23.74
4  2019-03-31 00:00:00+00:00          8848       224848       25.41
5  2019-06-30 00:00:00+00:00          6744       166749       24.73
6  2019-09-30 00:00:00+00:00          6588       164614       24.99
7  2019-12-31 00:00:00+00:00          7554       186289       24.66
8  2020-03-31 00:00:00+00:00         28886    

#Pegar os stop words da língua portuguesa usando NLTK

In [132]:
stop_words = set(stopwords.words('portuguese'))

In [133]:
type(stop_words)


set

In [134]:
stop_words

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houveríamos',
 'houvesse',


#Pré-processamento dos tweets

###Função para limpar os tweets

In [135]:
def cleaning(tweet):
  
  #remover pontuações
  def remove_punctuation(tweet):
      return re.sub(r'[^\w\s]','',tweet) 

  #remocao de plavras com mais de 3 ocorrencias seguidas
  def mais_de_3_ocorrencias_seguidas(word):  
    let_ant = word[0]
    count = 0
    mais_de_3 = False;
    for w in word:
      count = count + 1 if w == let_ant else 1
      let_ant = w  
      if count > 3: 
        mais_de_3 = True 
        break  
    return mais_de_3   

  #remover acentuações
  def remove_emphasis(token):
        return unidecode(remove_punctuation(token))

  stop_words_list = [remove_emphasis(st_word) for st_word in stop_words]
 
  #conversão de string para minúsculas, remoção de pontuação, palavras que iniciam com @, #, stopwords ou com tamanho < 2 e se forem diferentes de ht.
  def remove_undesirable_words(tweet):
      new_list_ = []
      tweet = ' '.join(str(word).lower() for word in tweet.split() if not word.startswith('@') and not word.startswith('#'))
     
      for token in wordpunct_tokenize(tweet):
         if token in stop_words_list or (len(token) < 3 and token != 'ht') or mais_de_3_ocorrencias_seguidas(token): continue
         new_list_.append(remove_emphasis(token))
      return ' '.join(new_list_)

  #remover urls
  def remove_url(tweet_without_sw):
      return re.sub(r"http\S+", "", tweet_without_sw)

  #remover palavras que contem ou que são números
  def remove_numeric(tweet):
    return re.sub(r'\d+', '', tweet)

  tweet = remove_url(tweet)
  tweet = remove_numeric(tweet)
  tweet = remove_undesirable_words(tweet)
  return tweet 



###Exemplo de tweet antes da aplicação da função de limpeza

In [136]:
string_example = "No Rio de Janeiro, em 2021, <p> o nao @senhor vai #investir, em 2022, na educação da Venezuela, Argentina, Haiti,cuba.e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso ?????. https://t.co/jmVSYbsG2X"

In [137]:
filtered_tweet = cleaning(string_example)

###Exemplo de tweet depois da aplicação da função de limpeza

In [138]:
filtered_tweet

'rio janeiro vai educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez'

###Aplicação da função de limpeza dos tweets

In [139]:
tweets_pre_proc = tweets_orig.copy()
tweets_pre_proc['text_preprocessed'] = tweets_orig['text'].map(lambda tweet: cleaning(tweet))

In [140]:
tweets_orig.text.head().tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???']

In [141]:
tweets_pre_proc.text_preprocessed.head().tolist()


['paqueta haitiano',
 'haiti desgraca fato erros anulam acertos fala sobre guerra ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto saara haiti',
 'entramos brics invadimos haiti mando bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia china inimigos mortais vai funcionar ']

###Remover tweets com menos de 3 palavras

In [144]:
tweets_pre_proc = tweets_pre_proc.loc[tweets_pre_proc['text_preprocessed'].str.split().str.len() > 3]

In [145]:
tweets_pre_proc.shape

(171087, 6)

###Remover os tweets duplicados

In [148]:
tweets_pre_proc.drop_duplicates(subset = ["text_preprocessed"], keep='first', inplace=True)

###Quantidade de tweets após a remoção dos tweets duplicados

In [149]:
tweets_pre_proc.shape

(150264, 6)

In [150]:
tweets_orig.text.head().tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???']

In [151]:
tweets_pre_proc.text_preprocessed.head().tolist()

['haiti desgraca fato erros anulam acertos fala sobre guerra ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto saara haiti',
 'entramos brics invadimos haiti mando bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia china inimigos mortais vai funcionar ',
 'haitiano entendi nada falavam']

###Estatística dos tweets após o pré-processamento:

In [152]:
tweets_stats(tweets_pre_proc, 'pre')

*****Agrupado por ano****
   Year  Count tweets  Count words  Mean_words
0  2018         18202       247212       13.58
1  2019         24449       364905       14.93
2  2020         47530       650724       13.69
3  2021         48733       704287       14.45
4  2022         11350       170537       15.03


*****Agrupamento trimestral****
                        Date  Count tweets  Count words  Mean_words
0  2018-03-31 00:00:00+00:00          4796        63502       13.24
1  2018-06-30 00:00:00+00:00          4443        58990       13.28
2  2018-09-30 00:00:00+00:00          4574        61020       13.34
3  2018-12-31 00:00:00+00:00          4349        63126       14.52
4  2019-03-31 00:00:00+00:00          7341       111249       15.15
5  2019-06-30 00:00:00+00:00          5431        80881       14.89
6  2019-09-30 00:00:00+00:00          5620        82266       14.64
7  2019-12-31 00:00:00+00:00          6059        90544       14.94
8  2020-03-31 00:00:00+00:00         24600    

###Lematização e stemização

In [153]:
pln = spacy.load('pt_core_news_sm')

###Bigramas

In [154]:
def bigrams(df, option):
  data = df.text_lematizados if option =='lem' else df.text_stematizados
  docs = [doc.split() for doc in data]
  bigram = Phrases(docs, min_count=3)
  for i in range(len(docs)):
    docs[i] = ' '.join(bigram[docs[i]])
  return docs  

###Lematização

In [155]:
string_lem_example = 'lixo tiver nada sou capacetes brancos ajudaram haiti pare besteira ajudou haiti fim exercito capacetes brancos resolvem problema presos campos concentracao argentina criou'

In [164]:
def lemmatization(tweet):
  document = pln(tweet)
  return ' '.join([str(token.lemma_).lower() for token in document])

In [157]:
tweet_lem_example = lemmatization(string_lem_example)

In [158]:
tweet_lem_example

'lixo ter nada ser capacete branco ajudar Haiti pare besteira ajudar Haiti fim exercito capacete branco resolver problema prender Campos concentracao Argentina criar'

In [165]:
tweets_pre_proc['text_lematizados'] = tweets_pre_proc['text_preprocessed'].map(lambda tweet: lemmatization(tweet))

In [166]:
tweets_pre_proc.text_lematizados.head().tolist()

['haiti desgracar fato erro anular acerto falar sobre guerra ucrania vir posicionar bem criticar postura zelensky',
 'senhor ir investir educacao venezuela argentina haiti cuba etc paise comunista porque aqui brasil senhor nunca fazer',
 'nada mao voce virar mistura deserto saara haiti',
 'entra brics invadir haiti mar bush lula dizer ir fortalecer brics fazer palestra parlamento europeu este tratar russia china inimigo mortal ir funcionar',
 'haitiano entendi nada falar']

In [167]:
tweets_pre_proc['bigrams_lem'] = bigrams(tweets_pre_proc, 'lem')

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [169]:
tweets_pre_proc.bigrams_lem.head().tolist()

['haiti desgracar fato erro anular acerto falar sobre guerra_ucrania vir posicionar bem criticar postura zelensky',
 'senhor ir investir_educacao venezuela argentina haiti cuba etc paise comunista porque aqui brasil senhor nunca fazer',
 'nada mao voce virar_mistura deserto saara haiti',
 'entra brics invadir haiti mar_bush lula dizer ir fortalecer brics fazer palestra parlamento europeu este tratar russia_china inimigo mortal ir funcionar',
 'haitiano entendi_nada falar']

###Stematização

In [170]:
string_stem_example = 'capacetes brancos ajudaram haiti pare besteira ajudou haiti fim exercito capacetes brancos resolvem problema presos campos concentracao argentina criou'

In [171]:
def stemming(tweet):
  document = pln(tweet)
  stemmer = nltk.stem.RSLPStemmer()
  return ' '.join([str(stemmer.stem(token.text)).lower() for token in document])

In [172]:
string_stem_example = stemming(string_stem_example)

In [173]:
string_stem_example

'capacet branc ajud hait par beste ajud hait fim exercit capacet branc resolv problem pres camp concentraca argentin cri'

In [174]:
tweets_pre_proc['text_stematizados'] = tweets_pre_proc['text_preprocessed'].map(lambda tweet: stemming(tweet))

In [175]:
tweets_pre_proc.text_stematizados.head().tolist()

['hait desgrac fat err anul acert fal sobr guerr ucran vem posicion bem critic post zelensky',
 'senh vai invest educaca venezuel argentin hait cub etc pais comun porqu aqu brasil senh nunc fez',
 'nad mao voc vir mist desert sa hait',
 'entr bric invad hait mand bush lul diz vai fortalec bric faz palestr parl europ est trat russ chin inimig mort vai funcion',
 'haiti entend nad fal']

In [176]:
tweets_pre_proc['bigrams_stem'] = bigrams(tweets_pre_proc,'stem')

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [177]:
tweets_pre_proc.bigrams_stem.head().tolist()

['hait desgrac fat err anul acert fal sobr guerr_ucran vem posicion bem critic post zelensky',
 'senh vai invest_educaca venezuel argentin hait cub etc pais comun porqu aqu brasil senh nunc fez',
 'nad mao voc vir_mist desert_sa hait',
 'entr bric invad hait mand bush lul diz vai fortalec bric faz palestr parl europ est trat russ_chin inimig mort vai funcion',
 'haiti entend_nad fal']

#Comparação de tweets originais, pré-processados, lematizados, **bigramados e **stematizados

###Tweets originais

In [178]:
tweets_orig.text.head().tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???']

### Tweets pré-processados

In [179]:
tweets_pre_proc.text_preprocessed.head().tolist()

['haiti desgraca fato erros anulam acertos fala sobre guerra ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto saara haiti',
 'entramos brics invadimos haiti mando bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia china inimigos mortais vai funcionar ',
 'haitiano entendi nada falavam']

###Tweets lematizados

In [180]:
tweets_pre_proc.text_lematizados.head().tolist()

['haiti desgracar fato erro anular acerto falar sobre guerra ucrania vir posicionar bem criticar postura zelensky',
 'senhor ir investir educacao venezuela argentina haiti cuba etc paise comunista porque aqui brasil senhor nunca fazer',
 'nada mao voce virar mistura deserto saara haiti',
 'entra brics invadir haiti mar bush lula dizer ir fortalecer brics fazer palestra parlamento europeu este tratar russia china inimigo mortal ir funcionar',
 'haitiano entendi nada falar']

###Tweets com bigramas

In [182]:
tweets_pre_proc.bigrams_lem.head().tolist()

['haiti desgracar fato erro anular acerto falar sobre guerra_ucrania vir posicionar bem criticar postura zelensky',
 'senhor ir investir_educacao venezuela argentina haiti cuba etc paise comunista porque aqui brasil senhor nunca fazer',
 'nada mao voce virar_mistura deserto saara haiti',
 'entra brics invadir haiti mar_bush lula dizer ir fortalecer brics fazer palestra parlamento europeu este tratar russia_china inimigo mortal ir funcionar',
 'haitiano entendi_nada falar']

###Tweets stematizados

In [183]:
tweets_pre_proc.text_stematizados.head().tolist()

['hait desgrac fat err anul acert fal sobr guerr ucran vem posicion bem critic post zelensky',
 'senh vai invest educaca venezuel argentin hait cub etc pais comun porqu aqu brasil senh nunc fez',
 'nad mao voc vir mist desert sa hait',
 'entr bric invad hait mand bush lul diz vai fortalec bric faz palestr parl europ est trat russ chin inimig mort vai funcion',
 'haiti entend nad fal']

##Criando o arquivo .csv dos tweets

In [184]:
df = pd.DataFrame(tweets_pre_proc)
df.to_csv('/content/drive/My Drive/Colab Notebooks/TCC/csv/pre_processed_tweets.csv')